# <a id='toc1_'></a>[JO 2024 project](#toc0_)

**Table of contents**<a id='toc0_'></a>    
- [JO 2024 project](#toc1_)    
  - [Prelude](#toc1_1_)    
  - [Imports](#toc1_2_)    
  - [Fonctions](#toc1_3_)    
  - [Data collect](#toc1_4_)    
    - [Extract one country data](#toc1_4_1_)    
    - [Extract All countries data](#toc1_4_2_)    
    - [Extract all data](#toc1_4_3_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

## <a id='toc1_1_'></a>[Prelude](#toc0_)

Work in progress . . .  

Summarize about the project :
- Data source : <a href="https://www.olympic.org/news">International Olympic Committee</a>
- Data extract from : <a href="http://olympanalyt.com/OlympAnalytics.php">olympanalyt.com - mail : sportsencyclo@gmail.com</a>

## <a id='toc1_2_'></a>[Imports](#toc0_)

In [147]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

## <a id='toc1_3_'></a>[Fonctions](#toc0_)

In [148]:
def GetHTML(url) :
    """
    Needs : url website
    Return : HTML text
    """
    response = requests.get(url)

    if response.status_code == 200 :
        print("Response OK, continue.")
    else :
        print("Access impossible.")
    
    return response.text

In [149]:
def Getdataframe(html) :

    """
    Needs : HTML text 
            with response = requests.get(url).text
    Return : Dataframe with athlete AND sport description : medal, country, athlete_name, games, sport, event
    Requirement : Must to apply on website like : adress[COUNTRY]adress
                i.e : http://olympanalyt.com/OlympAnalytics.php?param_pagetype=Medals&param_dbversion=&param_country=[COUNTRY]&param_games=ALL&param_sport=ALL
    """

    # Converting text (html) in soup (text parsed)
    soup = BeautifulSoup(html, "html5lib") #parse

    # Finding the correct table according to description
    table = soup.find("table", class_="simpletable", style="text-align:left;")

    # Retrieving the list (find_all) of all rows
    rows = table.find_all("tr")

    # Creating the table (future dataframe)
    athletes = []

    # For each row (= athlete)
    for row in rows:
        #describe_athlete = row.text.split() # Get the text
        cells = row.find_all("td") # Retrieving the list (find_all) of all cells in the row

        if len(cells) == 7: # If there are 7 cells then it's an athlete
            medal = cells[0].find('img')['title'][0]  # Medal title
            country = cells[1].find('img')['title']  # Athlete's country
            athlete_name = str(cells[2]).replace("<br/>","/").replace("</td>","").replace("<td>","") # Athlete's name, splitted by "/" if it's a team
            games = cells[3].find('img')['title']  # Olympic Games
            sport = cells[4].find('img')['title']  # Sport
            sex = cells[5].text.strip()  # Sex
            event = cells[6].text.strip()  # Event

            # Add the information to the list of athletes
            athletes.append({
                'medal': medal,
                'country': country,
                'athlete_name': athlete_name,
                'games': games,
                'sport': sport,
                'event': event,
                'sex' : sex
                })

        else: # Otherwise it's not an athlete, move to the next row
            next

    return pd.DataFrame(athletes)

url = "http://olympanalyt.com/OlympAnalytics.php?param_pagetype=Medals&param_dbversion=&param_country=FRA&param_games=ALL&param_sport=ALL"
# Try with one country - FRA (France)
html = GetHTML(url)

Getdataframe(html)

Response OK, continue.


,medal,country,athlete_name,games,sport,event,sex
0,1,France,Paul MASSON,Greece,Cycling Track,sprint individual,Men's
1,1,France,Paul MASSON,Greece,Cycling Track,1/3km time trial,Men's
2,1,France,Paul MASSON,Greece,Cycling Track,10km,Men's
3,1,France,Léon FLAMENG,Greece,Cycling Track,100km,Men's
4,1,France,Eugène-Henri GRAVELOTTE,Greece,Fencing,foil individual,Men's
...,...,...,...,...,...,...,...
884,2,France,SIMON Julia/FILLON MAILLET Quentin/CHEVALIER-B...,China,Biathlon,Relay mix,Mixed
885,2,France,LEDEUX Tess,China,Freestyle Skiing,Big air,Women's
886,2,France,TRESPEUCH Chloe,China,Snowboard,Snowboard Cross,Women's
887,3,France,FAIVRE Mathieu,China,Alpine Skiing,giant slalom,Men's


## <a id='toc1_4_'></a>[Data collect](#toc0_)

### <a id='toc1_4_1_'></a>[Extract one country data](#toc0_)

In [150]:
url = "http://olympanalyt.com/OlympAnalytics.php?param_pagetype=Medals&param_dbversion=&param_country=FRA&param_games=ALL&param_sport=ALL"

In [151]:
# Try with one country - FRA (France)
html = GetHTML(url)

Response OK, continue.


In [152]:
# Try to get the DF on one country - FRA (France)
df_medalists = Getdataframe(html)

df_medalists.sample(5)

,medal,country,athlete_name,games,sport,event,sex
692,2,France,Laetitia LE CORGUILLE,China,Cycling BMX Racing,racing individual,Women's
599,1,France,Северин Ванденхенде,Australia,Judo,57 - 63kg (half-middleweight),Women's
251,1,France,Louis CHAILLOT/Maurice PERRIN,United States,Cycling Track,2000m tandem,Men's
372,2,France,Claude ARABO,Japan,Fencing,sabre individual,Men's
452,3,France,Perrine PELEN,United States,Alpine Skiing,giant slalom,Women's


### <a id='toc1_4_2_'></a>[Extract All countries data](#toc0_)

Scrape countries codes to loop on each of them (website url modification)  
Link of countries codes : <a href="http://olympanalyt.com/OlympAnalytics.php?param_pagetype=RefCountries&param_dbversion=&param_country=CIV&param_games=ALL&param_sport=ALL"> Countries list</a>

In [153]:
# Countries list
url = "http://olympanalyt.com/OlympAnalytics.php?param_pagetype=RefCountries&param_dbversion=&param_country=CIV&param_games=ALL&param_sport=ALL"

html = GetHTML(url)

Response OK, continue.


In [154]:
# Parsing text in Beautifulsoup object
soup = BeautifulSoup(html, "html5lib")

In [155]:
# Select the right table
table = soup.find_all("table", class_="simpletable", style="text-align:center;")[1] # 2nd table
rows = table.find_all("tr")
countries = []

for row in rows[2:]: # ignore the column names
    data = row.text.split()

    if len(data) == 5 and len(data[0]) == 3:
        countries.append(
            {
                'Code':data[0], 
                'Country':data[1], 
                'Continent':data[2], 
                'Firstparticipation':data[3], 
                'Lastparticipation':data[4]
            }
        )
    else :
        next

df_countries = pd.DataFrame(countries)

In [156]:
df_countries.sample(5)

,Code,Country,Continent,Firstparticipation,Lastparticipation
104,MNE,Montenegro,Europe,2008,2022
72,ITA,Italy,Europe,1896,2022
115,NOR,Norway,Europe,1900,2022
23,BRA,Brazil,America,1920,2022
24,BUL,Bulgaria,Europe,1896,2022


### <a id='toc1_4_3_'></a>[Extract all data](#toc0_)

In [157]:
# Run scraping on all countries 
code_countries = df_countries["Code"]
name_countries = df_countries["Country"]
i = 0

for country, country_name in zip(code_countries, name_countries) :
    url = "http://olympanalyt.com/OlympAnalytics.php?param_pagetype=Medals&param_dbversion=&param_country=" + country + "&param_games=ALL&param_sport=ALL"
    html = GetHTML(url)
    df_temp = Getdataframe(html)
    if i == 0 :
        df_olympic = df_temp
    else : 
        df_olympic = pd.concat([df_olympic, df_temp], ignore_index=True)

    #Delete these codes after tries
    if i > 5 :
        break

    i += 1
    
    print("for : {}".format(country_name))

Response OK, continue.
for : Afghanistan
Response OK, continue.
for : Albania
Response OK, continue.
for : Algeria
Response OK, continue.
for : Andorra
Response OK, continue.
for : Angola
Response OK, continue.
for : Argentina
Response OK, continue.


In [158]:
df_olympic

,medal,country,athlete_name,games,sport,event,sex
0,3,Afghanistan,Rohullah NIKPAI,China,Taekwondo,- 58 kg,Men's
1,3,Afghanistan,Rohullah NIKPAI,Great Britain,Taekwondo,58 - 68 kg,Men's
2,3,Algeria,Mohamed ZAOUI,United States,Boxing,71 - 75kg (middleweight),Men's
3,3,Algeria,Mustapha MOUSSA,United States,Boxing,75 - 81kg (light-heavyweight),Men's
4,1,Algeria,Hassiba BOULMERKA,Spain,Athletics,1500m,Women's
...,...,...,...,...,...,...,...
109,2,Armenia,ARUTYUNYAN Migran,Brazil,Wrestling Greco-Roman,66kg,Men's
110,2,Armenia,MARTIROSYAN Simon,Japan,Weightlifting,109kg,Men's
111,2,Armenia,ALEKSANYAN Artur,Japan,Wrestling Greco-Roman,97kg,Men's
112,3,Armenia,BACHKOV Hovhannes,Japan,Boxing,57 - 63kg (lightweight),Men's


In [159]:
# Check chars non-ASCII return True
def contains_non_ascii(strings):
    """
    Return: result in Boolean
    """
    return any(ord(char) > 127 for char in strings)


# Filter non-ASCII & ASCII name
df_non_ascii = df_olympic[df_olympic['athlete_name'].apply(contains_non_ascii)]
df_ascii = df_olympic[~df_olympic['athlete_name'].apply(contains_non_ascii)]

In [160]:
# translate all Non-ASCII name in ASCII (latine) name
from transliterate import translit

df_non_ascii.athlete_name = df_non_ascii.athlete_name.apply(lambda s : translit(s, reversed=True))

C:\Users\Ygrek\AppData\Local\Temp\ipykernel_5880\2845671544.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_non_ascii.athlete_name = df_non_ascii.athlete_name.apply(lambda s : translit(s, reversed=True))


In [161]:
# Regroup both Dataframe, now all in Latine
df_final = pd.concat([df_ascii, df_non_ascii])

In [162]:
df_final.sample(10)

,medal,country,athlete_name,games,sport,event,sex
47,2,Argentina,Rufino RODRIGUEZ DE LA TORRE/Julio SIEBURGER/E...,Great Britain,Sailing,6m,Mixed
89,1,Argentina,Argentina,Brazil,Hockey,hockey,Men's
77,3,Argentina,Dzhordzhina Bardash,Greece,Swimming,400m individual medley,Women's
46,2,Argentina,Noemi SIMONETTO DE PORTELA,Great Britain,Athletics,long jump,Women's
55,2,Argentina,Humberto SELVETTI,Australia / Sweden,Weightlifting,+ 90kg (heavyweight),Men's
104,2,Armenia,Arsen JULFALAKYAN,Great Britain,Wrestling Greco-Roman,66 - 74kg,Men's
19,1,Argentina,Guillermo NAYLOR/Arturo KENNY/Juan NELSON/Enri...,France,Polo,polo,Men's
35,2,Argentina,Amado AZAR,United States,Boxing,66.68 - 72.57kg (middleweight),Men's
87,3,Argentina,Lucas CALABRESE/Juan DE LA FUENTE,Great Britain,Sailing,470 - Two Person Dinghy,Men's
50,1,Argentina,Eduardo GUERRERO/Tranquilo CAPPOZZO,Finland,Rowing,double sculls (2x),Men's


In [163]:
# Check if there are more than 1 athlete
def is_team(strings):
    """
    Return: result in Boolean
    """
    return len(strings.split("/")) > 1

# Team athletes
df_final_team = df_final[df_final["athlete_name"].apply(is_team)]

# Solo athlete
df_final_solo = df_final[~df_final["athlete_name"].apply(is_team)]

To do :
- Get simple sport list
- Get simple event list (weight, lenght, heigh, etc. split ?)
- Look for empty name (sometimes filled with country name)